In [ ]:
from grid.refine import *

In [ ]:
jittens.jobs.submit('python -c "from grid.refine import *" >logs.txt 2>&1', dir='.', resources={}, extras=['credentials.json'])

In [ ]:
jittens.manage.refresh()

In [ ]:
!tar -t ".jittens/archives/2021-02-14 20-23-32 every-nines.tar.gz"

In [ ]:
source = '.'
files = check_output(f'cd {quote(source)} && ag -g "" -l . </dev/null', shell=True, stderr=STDOUT)

In [ ]:
files.splitlines()